In [1]:
# libraries
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
# our modules
from preprocessing import *

In [2]:
torneo = pd.read_csv('data/2020.csv', 
                     encoding='utf-8-sig', 
                     dtype=DATA_TYPES,
                     parse_dates=['Date', 'WBD', 'LBD'])
players = {k : {} for k in set(torneo['Winner']).union(set(torneo['Loser'])) }
players

{'Bedene A.': {},
 'Tsonga J.W.': {},
 'Kukushkin M.': {},
 'Jaziri M.': {},
 'Kecmanovic M.': {},
 'Herbert P.H.': {},
 'Edmund K.': {},
 'Sandgren T.': {},
 'Rublev A.': {},
 'Barrere G.': {},
 'Tiafoe F.': {},
 'Andujar P.': {},
 'Moutet C.': {},
 'Cecchinato M.': {},
 'Thompson J.': {},
 'Djere L.': {},
 'Fucsovics M.': {},
 'Sonego L.': {},
 'Ymer M.': {},
 'Bublik A.': {},
 'Verdasco F.': {},
 'Krajinovic F.': {},
 'Ilkel C.': {},
 'Berankis R.': {},
 'Chardy J.': {},
 'Mannarino A.': {},
 'Raonic M.': {},
 'Wawrinka S.': {}}

In [ ]:
features_to_drop = ['Best of', 'Court','Surface']
X, Y = preprocess_data(2011, 2019, features_to_drop)

In [ ]:
# Build the model
rf = RandomForestClassifier(n_estimators=250,
                            bootstrap=True,
                            criterion='entropy',
                            max_features='log2',
                            max_depth=15,
                            n_jobs=-1)
rf.fit(X_train, Y_train)

In [ ]:
def simulation(model,
               atp = 1, 
               features_to_drop=[], 
               missing_values="drop", 
               drop_first=False):
    
    tournament = pd.read_csv("data/" + str(2020) + ".csv", encoding='utf-8-sig', dtype=DATA_TYPES)
    tournament = tournament[tournament['ATP'] == atp]
    # TODO filtrare per atp e round(primo)
    
    tournament_to_test = unify_data(tournament, features_to_drop, 'drop', drop_first)
    
    prediction = model.predict(tournament_to_test[tournament_to_test['Round'] == 0])
    accuracy = round(accuracy_score(y_true=np.ones(len(prediction), dtype=int), y_pred=prediction), 3)
    print("Accuracy: ", accuracy)
    
    firstR = tournament[tournament['Round'] == '1st Round']
    i = 0
    for j, match in firstR.iterrows():
        print('P1: ' + match['Winner'], 'P2: '+ match['Loser'], 'Wins:', prediction[i])
        i+=1

    #print(prediction)
    
    return tournament_to_test, X, Y
    

In [ ]:
T, X, Y = simulation(rf, 1, features_to_drop)

In [ ]:
#print(T.iloc[0],T.iloc[4], T.iloc[7], T.iloc[8])

Notare che il modello sbaglia la predizione quando anche le scommesse(Max e Avg) sbagliano(era più quotata la vittoria del giocatore che di fatto ha perso)
TODO: 
    - fare un decisore stupido basato sulle scommesse
    - provare a tenere B365, nel caso droppare Avg e Max
    
## Preparazione per Australian Open 2020
Scraping dei giocatori del primo round

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from preprocessing import uniformName

In [ ]:
columns = {
    'Series': object,
    'Court': object,
    'Surface': object,
    'Round': object,
    'Best of': int,
    'Winner': object,
    'Loser': object,
    'WRank': float,
    'LRank': float,
    'WPts': float,
    'LPts': float,
    'WElo': float,
    'WSurfElo': float,
    'WHand': object,
    'WBHand': float,
    'LElo': float,
    'LSurfElo': float,
    'LHand': object,
    'LBHand': float,
    'WEloCalc': float,
    'LEloCalc': float,
    'ProbaElo': float}


In [ ]:
driver = webdriver.Chrome()
driver.get('https://ausopen.com/draws#!mens-singles')
sleep(5)
firstRound = driver.find_element_by_class_name('columm-1')
matches = firstRound.find_elements_by_class_name('match-teams')
i = 1
for m in matches:
    print('Match ', i)
    players = m.find_elements_by_class_name('team-detail__players')
    for player in players:
        try:
            p = player.find_element_by_class_name('player-full-name').get_attribute('innerHTML')
        except Exception:
            p = 'Qualifier'
            
        print(p)
    i+=1